In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!ls "/content/drive/MyDrive"

'210_44K22.2_CP2_baocao - Trang Nguyễn Thị Minh.gdoc'
 abbreviations.xlsx
 Bontry.gsheet
 Bontry.xlsx
 Classroom
'Colab Notebooks'
'DASHBOARD DATACO.twbx'
'ĐỀ CƯƠNG SƠ BỘ CAPSTONE PROJECT1.gdoc'
 hasaki7000.xlsx
 HASAKIHEHEHE.sql
 OTHERS_MODEL.joblib
 PACKAGING_MODEL.joblib
 PRICE_MODEL.joblib
 QUALITY_MODEL.joblib
 SERVICE_MODEL.joblib
 STORE_MODEL.joblib
 tokenizer.pkl
'TranVanLoc_HoLeKhoiNguyen (3).zip'
 vietnamese-stopwords-1.txt
'vietnamese-stopwords (1).txt'
 Visualize
 word2vec_sentiment.model


In [ ]:
!pip install numpy==1.23.5 pandas==1.5.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 89.0 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 1.5.3 which is incompatible.
dask-expr 1.1.21 requires pandas>=2, but you have pandas 1.5.3 which is incompatible.
bigframes 2.4.0 requires numpy>=1.24.0, but you have numpy 1.23.5 which is incompatible.
tensorflow-decision-forests 1.11.0 requires tensorflow==2.18.0, but you have tensorflow 2.13.0 which is incompatible.
tsfresh 0.21.0 requires scipy>=1.14.0; python_version >= "3.10", but you have scipy 1.13.1 which is incompatible.
db-dtypes 1.4.3 requires numpy>=1.24.0, but you have numpy 1.23.5 which is incompatible.
dask-c

In [ ]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Embedding, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import load_model
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split, KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from joblib import dump, load
from sklearn.preprocessing import label_binarize

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [ ]:
import re
!pip install pyvi
from pyvi import ViTokenizer
!pip install emoji
import emoji
# Load các thành phần đã lưu
w2v_model = Word2Vec.load('/content/drive/MyDrive/word2vec_sentiment.model')
tokenizer_load = load('/content/drive/MyDrive/tokenizer.pkl')
QUALITY_model = load('/content/drive/MyDrive/QUALITY_MODEL.joblib')
SERVICE_model = load('/content/drive/MyDrive/SERVICE_MODEL.joblib')
OTHERS_model = load('/content/drive/MyDrive/OTHERS_MODEL.joblib')
STORE_model = load('/content/drive/MyDrive/STORE_MODEL.joblib')
PACKAGING_model = load('/content/drive/MyDrive/PACKAGING_MODEL.joblib')
PRICE_model = load('/content/drive/MyDrive/PRICE_MODEL.joblib')

# Tiền xử lý
def preprocess_text(text, abbreviation_dict, stopwords):
    # Kiểm tra nếu input không phải là chuỗi, trả về chuỗi rỗng
    if not isinstance(text, str):
        return ""
    # 1. Chuyển về chữ thường
    text = text.lower()
    # 2. Giải mã chữ viết tắt
    for abbreviation, meaning in abbreviation_dict.items():
        text = re.sub(r'\b' + re.escape(abbreviation) + r'\b', meaning, text)
    # 3. Chuyển biểu tượng cảm xúc thành văn bản
    text = emoji.demojize(text)
    # 4. Loại bỏ ký tự đặc biệt
    text = re.sub(r"[!@#$[]()]", "", text)
    text = re.sub(r'[^\w\s]', '', text)
    # 5. Tách từ và loại bỏ stopwords
    tokenized_text = ViTokenizer.tokenize(text)
    tokens = [word for word in tokenized_text.split() if word not in stopwords]
    # 6. Trả về danh sách các từ đã xử lý
    return tokens

# Đọc danh sách chữ viết tắt
abbreviation_path = "/content/drive/MyDrive/abbreviations.xlsx"
abbs_df = pd.read_excel(abbreviation_path, engine='openpyxl')
abbreviation_dict = dict(zip(abbs_df['abbreviation'].astype(str), abbs_df['meaning']))

# Đọc danh sách stopwords
stopwords_path = "/content/drive/MyDrive/vietnamese-stopwords (1).txt"
with open(stopwords_path, "r", encoding="utf-8") as f:
    stopwords = f.read().split("\n")

# Hàm chuyển đổi dữ liệu sang vector đặc trưng bằng Word2Vec
def vectorize_comment(comment, model, vector_dim):
    words = comment
    word_vectors = [model.wv[word] for word in words if word in list(w2v_model.wv.index_to_key)]
    if len(word_vectors) == 0:
        return np.zeros(vector_dim)  # Nếu không có từ nào trong mô hình Word2Vec
    return np.mean(word_vectors, axis=0)


# Pipeline dự đoán
def predict_sentiment(input_text):
    processed_text_RF = preprocess_text(input_text,abbreviation_dict, stopwords)
    processed_text_NN = preprocess_text(input_text,abbreviation_dict, stopwords)
    # Ghép các từ đã token hóa thành một chuỗi duy nhất
    processed_text_joined = " ".join(processed_text_NN)
    # Chuyển đổi sang chuỗi số bằng tokenizer
    new_sequences = tokenizer_load.texts_to_sequences([processed_text_joined])  # Đảm bảo đầu vào là danh sách chuỗi
    # Chuyển dữ liệu thành padded sequences
    max_length = 100
    new_padded = pad_sequences(new_sequences, maxlen=max_length, padding='post', truncating='post')

    input_vector = vectorize_comment(processed_text_RF, w2v_model, vector_dim=150)

    QUALITY = QUALITY_model.predict([input_vector])[0]
    SERVICE_pred = SERVICE_model.predict(new_padded)
    SERVICE = np.argmax(SERVICE_pred)
    OTHERS = OTHERS_model.predict([input_vector])[0]
    STORE = STORE_model.predict([input_vector])[0]
    PACKAGING = PACKAGING_model.predict([input_vector])[0]
    PRICE_pred = PRICE_model.predict(new_padded)
    PRICE = np.argmax(PRICE_pred)
    return reverse_label_mapping[QUALITY],reverse_label_mapping[SERVICE],reverse_label_mapping[OTHERS],reverse_label_mapping[STORE],reverse_label_mapping[PACKAGING],reverse_label_mapping[PRICE]

# Mapping nhãn
label_mapping = {"positive": 2, "neutral": 1, "negative": 0}
reverse_label_mapping = {v: k for k, v in label_mapping.items()}

# Ví dụ
input_text = "Tôi thấy sản phẩm này cũng dùng được, bạn tư vấn rất nhiệt tình tuy nhiên bao bì hơi xấu cần cải thiện nội dung trên bao bì"
QUALITY, SERVICE, OTHERS, STORE, PACKAGING, PRICE = predict_sentiment(input_text)
print(f"Input: {input_text}")
print(f"QUALITY: {QUALITY}")
print(f"SERVICE: {SERVICE}")
print(f"OTHERS: {OTHERS}")
print(f"STORE: {STORE}")
print(f"PACKAGING: {PACKAGING}")
print(f"PRICE: {PRICE}")